In [110]:
import pandas as pd
import nltk
import re
from nltk import sent_tokenize,word_tokenize,PorterStemmer
from nltk.corpus import stopwords
stopWords =set(stopwords.words("english"))
ps=PorterStemmer()
import math

In [15]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Siddhi\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [137]:
text= open("C:/Users/Siddhi/Desktop/data.txt","r",encoding='utf-8')

In [138]:
text1=str(text.read())

In [139]:
sent="samir likes to play cricket everyday because he likes cricket so much. ajay is going to his home to play cricket";

In [140]:
def frequency_table(text):
    words=word_tokenize(text)
    word_freq=dict()
    for x in words:
        x=ps.stem(x.lower())
        if(x not in stopWords) and (len(x)>2):
            if x not in word_freq.keys():
                word_freq[x]=1
            else:
                word_freq[x]+=1
    return word_freq

In [141]:
frequency_table(sent)

{'samir': 1,
 'like': 2,
 'play': 2,
 'cricket': 3,
 'everyday': 1,
 'becaus': 1,
 'much': 1,
 'ajay': 1,
 'home': 1}

In [142]:
def frequency_matrix(sentences):
    frequency_matrix=dict()
    for sent in sent_tokenize(sentences):
        frequency_matrix[sent[:15]]=frequency_table(sent)
    return frequency_matrix

In [143]:
freq_matrix=frequency_matrix(sent)
freq_matrix

{'samir likes to ': {'samir': 1,
  'like': 2,
  'play': 1,
  'cricket': 2,
  'everyday': 1,
  'becaus': 1,
  'much': 1},
 'ajay is going t': {'ajay': 1, 'home': 1, 'play': 1, 'cricket': 1}}

In [144]:
 def tf_matrix(freq_matrix):
        tf_matrix={}
        for sent,freq_table in freq_matrix.items():
            tf_table={}
            ft_len=len(freq_table)
            for Dict in freq_table.keys():
                tf_table[Dict]=round(freq_table[Dict]/ft_len,2)
            tf_matrix[sent]=tf_table
        return tf_matrix

In [145]:
tf_matrix1=tf_matrix(freq_matrix)
tf_matrix1

{'samir likes to ': {'samir': 0.14,
  'like': 0.29,
  'play': 0.14,
  'cricket': 0.29,
  'everyday': 0.14,
  'becaus': 0.14,
  'much': 0.14},
 'ajay is going t': {'ajay': 0.25,
  'home': 0.25,
  'play': 0.25,
  'cricket': 0.25}}

In [146]:
def doc_frequency(freq_matrix):
    doc_table={}
    for sent,tf_table in freq_matrix.items():
        for word in tf_table.keys():
            if word not in doc_table.keys():
                doc_table[word]=1
            else:
                doc_table[word]+=1
    return doc_table

In [147]:
doc_matrix= doc_frequency(freq_matrix)
doc_matrix

{'samir': 1,
 'like': 1,
 'play': 2,
 'cricket': 2,
 'everyday': 1,
 'becaus': 1,
 'much': 1,
 'ajay': 1,
 'home': 1}

In [148]:
def idf_matrix(freq_matrix,doc_per_word):
    total_doc=len(freq_matrix.keys())
    tf_matrix={}
    for sent,tf_table in freq_matrix.items():
        freq_table={}
        for word in tf_table.keys():
            freq_table[word]=round(math.log10(total_doc/doc_per_word[word]),2)
        tf_matrix[sent]=freq_table
    return tf_matrix

In [149]:
idf1_matrix=idf_matrix(freq_matrix,doc_matrix)
idf1_matrix

{'samir likes to ': {'samir': 0.3,
  'like': 0.3,
  'play': 0.0,
  'cricket': 0.0,
  'everyday': 0.3,
  'becaus': 0.3,
  'much': 0.3},
 'ajay is going t': {'ajay': 0.3, 'home': 0.3, 'play': 0.0, 'cricket': 0.0}}

In [150]:
def tf_idf_matrix(tf_matrix,idf_matrix):
    tf_idf_matrix={}
    for (sent1,ft_table1),(sent2,ft_table2) in zip(tf_matrix.items(),idf_matrix.items()):
        freq_table={}
        for(word1,value1),(word2,value2) in zip(ft_table1.items(),ft_table2.items()):
            freq_table[word1]=round(value1*value2,2)
        tf_idf_matrix[sent1]=freq_table
    return tf_idf_matrix

In [151]:
tf_idf_matrix1=tf_idf_matrix(tf_matrix1,idf1_matrix)
tf_idf_matrix1

{'samir likes to ': {'samir': 0.04,
  'like': 0.09,
  'play': 0.0,
  'cricket': 0.0,
  'everyday': 0.04,
  'becaus': 0.04,
  'much': 0.04},
 'ajay is going t': {'ajay': 0.07, 'home': 0.07, 'play': 0.0, 'cricket': 0.0}}

In [152]:
def score_sentence(tf_idf_matrix):
    sent_score={}
    for sent,ft_table in tf_idf_matrix.items():
        for word,value in ft_table.items():
            if sent not in sent_score.keys():
                sent_score[sent]=value
            else:
                sent_score[sent]+=value
    return sent_score

In [153]:
sent_score=score_sentence(tf_idf_matrix1)
sent_score

{'samir likes to ': 0.25, 'ajay is going t': 0.14}

In [154]:
def average_score(sent_score):
    sum_sent=0
    for value in sent_score.values():
        sum_sent+=value
    avg_score=round(sum_sent/len(sent_score),2)
    return avg_score

In [155]:
def get_summary(sentences,sentence_score,threshold):
    summary=""
    for sent in sentences:
        if(sent[:15] in sentence_score) and sentence_score[sent[:15]]>=threshold:
            summary+=" "+sent
    return summary

In [156]:
def lets_summarize(text):
    sentences=sent_tokenize(text)
    freq_matrix=frequency_matrix(text)
    tf_matrix1=tf_matrix(freq_matrix)
    doc_matrix=doc_frequency(freq_matrix)
    idf1_matrix=idf_matrix(freq_matrix,doc_matrix)
    tf_idf_matrix1=tf_idf_matrix(tf_matrix1,idf1_matrix)
    sent_score=score_sentence(tf_idf_matrix1)
    threshold=average_score(sent_score)
    summary=get_summary(sentences,sent_score,1.15*threshold)
    return summary

In [157]:
lets_summarize(text1)

' Artificial intelligence (AI), sometimes called machine intelligence, is intelligence demonstrated by machines, unlike the natural intelligence displayed by humans and animals. A quip in Tesler\'s Theorem says "AI is whatever hasn\'t been done yet." Artificial intelligence was founded as an academic discipline in 1955, and in the years since has experienced several waves of optimism, followed by disappointment and the loss of funding (known as an "AI winter"), followed by new approaches, success and renewed funding.'